# <B> Setup for api-gateway with lambda </B>

## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## 0. Install requirements

In [2]:
%%sh
bash ./dependency/setup.sh

~/SageMaker/api-gateway-lambda/dependency ~/SageMaker/api-gateway-lambda

=== Environment Settting for Dynamic AB-Testing with MLOps ===
    WORKDIR: /home/ec2-user/SageMaker/api-gateway-lambda/dependency 

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com

=== Done ===
~/SageMaker/api-gateway-lambda


## 1. Set roles

In [3]:
from utils.iam import iam_handler
from sagemaker import get_execution_role

In [4]:
iam = iam_handler()
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20221206T163436


In [5]:
strPrefix = "API-GATEWAY-LAMBDA-DEMO-"

### 2.1 Attach IAM polich to sagemaker execution role (<b>with console</b>)
> step 1. IAM console 로 이동    
>
> step 2. 왼쪽 네비게이터에서  "Role" 선택  
> ![nn](images/Role.png)  
>
> step 3. SageMaker Execution Role 검색 후 role 선택 (상위 cell output 참조)  
> ![nn](images/search-by-rolename.png)  
>
> step 4. "attach policies" 메뉴 선택 
> ![nn](images/attach-policy-menu.png)  
>
> step 5. "IAMFullAccess" policy 검색 후 attach 
> ![nn](images/attach-policy.png) 

### 2.2 Attach policy - sagemaker execution role
> SSM  
> Lambda  
> API-GATEWAY  
> DynamoDB

In [6]:
listPolicyArn=[
    "arn:aws:iam::aws:policy/AmazonSSMFullAccess",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess",
    "arn:aws:iam::aws:policy/AmazonAPIGatewayAdministrator",
    "arn:aws:iam::aws:policy/AmazonDynamoDBFullAccess",
]
for strPolicyArn in listPolicyArn: iam.attach_policy(strSageMakerRoleName, strPolicyArn)

### 2.3 role-lambda
> SageMaker  
> DynamoDB   
> LambdaBasicExecution  

In [7]:
listService = ["lambda.amazonaws.com"]
strRoleName = "".join([strPrefix, "LambdaRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole",
    "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess",
    "arn:aws:iam::aws:policy/AmazonDynamoDBFullAccess"
]
strDescription="Role for Lambda to call SageMaker functions'"

In [8]:
lambdaRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)
print (f"LAMBDA-ROLE-ARN: {lambdaRoleArn}")

== CREATE ROLE ==
  Service name: ['lambda.amazonaws.com'], 
  Role name: API-GATEWAY-LAMBDA-DEMO-LambdaRole, 
  Policys: ['arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole', 'arn:aws:iam::aws:policy/AmazonSageMakerFullAccess', 'arn:aws:iam::aws:policy/AmazonDynamoDBFullAccess'], 
  Role ARN: arn:aws:iam::419974056037:role/API-GATEWAY-LAMBDA-DEMO-LambdaRole
== COMPLETED ==
LAMBDA-ROLE-ARN: arn:aws:iam::419974056037:role/API-GATEWAY-LAMBDA-DEMO-LambdaRole


### 2.4 role-apigateway
> AmazonAPIGatewayPushToCloudWatchLogs

In [17]:
listService = ["apigateway.amazonaws.com"]
strRoleName = "".join(["apiGatewayRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/service-role/AmazonAPIGatewayPushToCloudWatchLogs",
]
strDescription="Role for API-Gateway to get permission to CloudWatch'"

In [20]:
apiGatewayRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)
print (f"API-GATEWAY-ROLE-ARN: {lambdaRoleArn}")

== CREATE ROLE ==
  Role Name: [apiGatewayRole] is already exist!!, so, this will be deleted and re-created.
{'ResponseMetadata': {'RequestId': 'c31a63da-6fa6-4b71-843c-a5c53fc4ce8c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c31a63da-6fa6-4b71-843c-a5c53fc4ce8c', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Tue, 13 Dec 2022 02:44:56 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/service-role/AmazonAPIGatewayPushToCloudWatchLogs
  Service name: ['apigateway.amazonaws.com'], 
  Role name: apiGatewayRole, 
  Policys: ['arn:aws:iam::aws:policy/service-role/AmazonAPIGatewayPushToCloudWatchLogs'], 
  Role ARN: arn:aws:iam::419974056037:role/apiGatewayRole
== COMPLETED ==
API-GATEWAY-ROLE-ARN: arn:aws:iam::419974056037:role/API-GATEWAY-LAMBDA-DEMO-LambdaRole


### 2.5 Enable CloudWatch Logs in API Gateway REST API
> step 1. api-gateway setting 화면 접속 (https://console.aws.amazon.com/apigateway/home#/settings)
>
> step 2. "CloudWatch log role ARN"에 2.4에서 만든 apigatewat role ("apiGatewayRole") ARN 입력
>> 상위 cell output 참고


## 2. Set default parameters

In [21]:
import boto3
import sagemaker
from utils.ssm import parameter_store

In [24]:
pm = parameter_store(strRegionName)
strRegionName=boto3.Session().region_name

In [25]:
pm.put_params(key="PREFIX", value=strPrefix, overwrite=True)
pm.put_params(key="".join([strPrefix, "REGION"]), value=strRegionName, overwrite=True)
pm.put_params(key="".join([strPrefix, "ACCOUNT-ID"]), value=boto3.client("sts").get_caller_identity().get("Account"), overwrite=True)
pm.put_params(key="".join([strPrefix, "BUCKET"]), value=sagemaker.Session().default_bucket(), overwrite=True)
pm.put_params(key="".join([strPrefix, "SAGEMAKER-ROLE-ARN"]), value=get_execution_role(), overwrite=True)
pm.put_params(key="".join([strPrefix, "LAMBDA-ROLE-ARN"]), value=lambdaRoleArn, overwrite=True)

'Store suceess'